# Data types á la carte [Swierstra]

## 2. Fixing the expression problem

In [1]:
data Expr f = In (f (Expr f)) -- same thing as Fix in recursion schemes

-- things they don't explain in this paper: the "magic" in the above definition.
-- how it solves the arbitrary nested type problem

data Val e = Val Int
data Add e = Add e e

{-# LANGUAGE TypeOperators #-}

infixr 6 :+: -- I think this line is not in the paper but without it it doesn't work

data (f :+: g) e = Inl (f e) | Inr (g e)

-- `Expr (Val :+: Add)`: an expression built from subexpressions of type `Val` or `Add`

addExample :: Expr (Val :+: Add)
addExample = In (Inr (Add (In (Inl (Val 118))) (In (Inl (Val 1219)))))

-- fixes the expression problem b/c you can add new types and new functions easily?
-- how to eval an expression like this?

## 3. Evaluation

In [2]:
instance Functor Val where
    fmap f (Val x) = Val x
    
instance Functor Add where
    fmap f (Add l r) = Add (f l) (f r) 
    
instance (Functor f, Functor g) => Functor (f :+: g) where
    fmap f (Inl l) = Inl (fmap f l)
    fmap f (Inr r) = Inr (fmap f r)
    
foldExpr :: Functor f => (f a -> a) -> Expr f -> a -- same as cata in recursion schemes, I think
foldExpr f (In t) = f (fmap (foldExpr f) t)

* `f a -> a` is an algebra: something that does one step of the recursion using the results of the recursive calls
* Idea: define and assemble algebras from individual cases (e.g. `Val` or `Add`) in order to define the algebra for the coproduct (e.g. `Val :+: Add`). We can use the type class system for this and teach it how to compose algebras.

In [3]:
-- A separate class for the algebra we aim to define:
class Functor f => Eval f where
    evalAlgebra :: f Int -> Int
    
instance Eval Val where
    evalAlgebra (Val x) = x
    
instance Eval Add where
    evalAlgebra (Add x y) = x + y

instance (Eval f, Eval g) => Eval (f :+: g) where
    evalAlgebra (Inl l) = evalAlgebra l
    evalAlgebra (Inr r) = evalAlgebra r
    
eval :: Eval f => Expr f -> Int
eval expr = foldExpr evalAlgebra expr

eval addExample

1337

## 4. Automating injections

A first attempt:

In [4]:
val :: Int -> Expr Val
val x = In (Val x)

infixl 6 <+>

(<+>) :: Expr Add -> Expr Add -> Expr Add
x <+> y = In (Add x y)

val 1 <+> val 2 -- d'oh

<interactive>:1:1: error:
    • Couldn't match type ‘Val’ with ‘Add’
      Expected type: Expr Add
        Actual type: Expr Val
    • In the first argument of ‘(<+>)’, namely ‘val 1’
      In the expression: val 1 <+> val 2
      In an equation for ‘it’: it = val 1 <+> val 2
<interactive>:1:11: error:
    • Couldn't match type ‘Val’ with ‘Add’
      Expected type: Expr Add
        Actual type: Expr Val
    • In the second argument of ‘(<+>)’, namely ‘val 2’
      In the expression: val 1 <+> val 2
      In an equation for ‘it’: it = val 1 <+> val 2

Idea: make signatures less rigid

```haskell
(⊕) :: (Add :≺: f ) ⇒ Expr f → Expr f → Expr f
val :: (Val :≺: f ) ⇒ Int → Expr f
```

> The constraint `sub :≺: sup` should only be satisfied if there is some injection from `sub a` to `sup a`.

Is `:≺:` a natural transformation?

In [5]:
{-# LANGUAGE MultiParamTypeClasses #-}

-- Idea: rather than writing `Inr` or `Inl` by hand,the 
-- injections can be inferred using instances of this class:
class (Functor sub, Functor sup) => sub :<: sup where
    inj :: sub a -> sup a
    
{-# LANGUAGE FlexibleInstances #-}

-- "Constructors" that use the types:
    
-- reflexivity: something can be injected to itself
-- base case of the recursive search?
instance Functor f => f :<: f where
    inj = id

-- "if f is true then f or g is true". 
-- Inject anything into any coproduct that contains it explicitly
instance {-# OVERLAPPING #-} (Functor f , Functor g) => f :<: (f :+: g) where
    inj = Inl

-- Inject anything into any coproduct that contains something it can be injected into
instance (Functor f , Functor g, Functor h, f :<: g) => f :<: (h :+: g) where
    inj = Inr . inj

In [6]:
-- defining smart constructors

inject :: (g :<: f) => g (Expr f) -> Expr f
inject = In . inj

{-# LANGUAGE FlexibleContexts #-}

val :: (Val :<: f) => Int -> Expr f
val = inject . Val

infixl 6 <+>

(<+>) :: (Add :<: f) => Expr f -> Expr f -> Expr f
l <+> r = inject (Add l r)

x = val 30000 <+> val 1330 <+> val 7 :: Expr (Val :+: Add)

eval x

31337

## 5. Examples

### Adding a new type

In [7]:
data Mul x = Mul x x

instance Functor Mul where
    fmap f (Mul l r) = Mul (f l) (f r)
    
instance Eval Mul where
    evalAlgebra (Mul x y) = x * y
    
infixl 7 <&> -- don't use <*> to avoid collisions with Applicative

(<&>) :: (Mul :<: f) => Expr f -> Expr f -> Expr f
l <&> r = inject (Mul l r)

x = val 80 <&> val 5 <+> val 4 :: Expr (Mul :+: Add :+: Val)
eval x

404

### Adding a new function

Objective: Render expressions

A first candidate:

In [8]:
class Render f where
    render :: f (Expr f) -> String

> The type of render, however, is not general enough. To see this, consider the instance definition for `Add`. We would like to make recursive calls to the subtrees, which themselves might be values, for instance. The above type for render, however, requires that all subtrees of `Add` are themselves additions. Clearly this is undesirable.

Why `Eval` doesn't have this problem?

A better choice:

In [9]:
class Render f where
    render :: Render g => f (Expr g) -> String
    
pretty :: Render f => Expr f -> String
pretty (In t) = render t

instance Render Val where
    render (Val i ) = show i

instance Render Add where
    render (Add x y) = "(" ++ pretty x ++ " + " ++ pretty y ++ ")"

instance Render Mul where
    render (Mul x y) = "(" ++ pretty x ++ " * " ++ pretty y ++ ")"

instance (Render f , Render g) => Render (f :+: g) where
    render (Inl x) = render x
    render (Inr y) = render y
    
x = val 80 <&> val 5 <+> val 4 :: Expr (Val :+: Add :+: Mul)

pretty x

"((80 * 5) + 4)"

Digression (?):

`inj` in the `(:<:)` has a partial inverse. We could have defined the class like this:

In [66]:
class (Functor sub, Functor sup) => sub :<: sup where
    inj :: sub a -> sup a
    prj :: sup a -> Maybe (sub a)
    
-- redefining instances:

instance Functor f => f :<: f where
    inj = id
    prj = Just

instance {-# OVERLAPPING #-} (Functor f , Functor g) => f :<: (f :+: g) where
    inj = Inl
    prj (Inl x) = Just x
    prj (Inr _) = Nothing

instance (Functor f , Functor g, Functor h, f :<: g) => f :<: (h :+: g) where
    inj = Inr . inj
    prj (Inl h) = Nothing
    prj (Inr g) = prj g
    
match :: (g :<: f ) => Expr f -> Maybe (g (Expr f ))
match (In t) = prj t

distr :: (Add :<: f , Mul :<: f ) => Expr f -> Maybe (Expr f )
distr t = do
    Mul a b <- match t
    Add c d <- match b
    return $ (a <&> c) <+> (a <&> d ) -- doesn't compile :(

<interactive>:21:14: error:
    • Could not deduce (Add Ghci66.:<: f) arising from a use of ‘<+>’
      from the context: (Add :<: f, Mul :<: f)
        bound by the type signature for:
                   distr :: (Add :<: f, Mul :<: f) => Expr f -> Maybe (Expr f)
        at <interactive>:17:1-62
    • In the second argument of ‘($)’, namely ‘(a <&> c) <+> (a <&> d)’
      In a stmt of a 'do' block: return $ (a <&> c) <+> (a <&> d)
      In the expression:
        do { Mul a b <- match t;
             Add c d <- match b;
             return $ (a <&> c) <+> (a <&> d) }
<interactive>:21:15: error:
    • Could not deduce (Mul Ghci66.:<: f) arising from a use of ‘<&>’
      from the context: (Add :<: f, Mul :<: f)
        bound by the type signature for:
                   distr :: (Add :<: f, Mul :<: f) => Expr f -> Maybe (Expr f)
        at <interactive>:17:1-62
    • In the first argument of ‘(<+>)’, namely ‘(a <&> c)’
      In the second argument of ‘($)’, namely ‘(a <&> c) <+> (a <&> d)’
      In a stmt of a 'do' block: return $ (a <&> c) <+> (a <&> d)

## Monads for free

In [63]:
data Term f a = Pure a
              | Impure (f (Term f a))
              
instance Functor f => Functor (Term f) where
    fmap f (Pure x)   = Pure (f x)
    fmap f (Impure t) = Impure (fmap (fmap f) t)
    
instance Functor f => Applicative (Term f) where
    pure = Pure
    (Pure f)    <*> (Pure a)    = Pure (f a)
    ff          <*> (Impure ta) = Impure (fmap (ff <*>) ta)
    (Impure tf) <*> a           = Impure (fmap (<*> a) tf)

instance Functor f => Monad (Term f) where
    return = Pure
    (Pure x)   >>= f = f x
    (Impure t) >>= f = Impure (fmap (>>= f) t)
    
data Zero a

type Identity a = Term Zero a

instance Functor Zero where
    fmap _ _ = undefined -- ???
    
myId :: a -> Identity a
myId = Pure

instance Show a => Show (Identity a) where
    show (Pure a) = show a

do x <- myId 2
   y <- myId 21
   return (x * y)

data One a = One

instance Functor One where
    fmap _ _ = One

type MyMaybe = Term One

myJust :: a -> MyMaybe a
myJust = Pure

myNothing :: MyMaybe a
myNothing = Impure One

instance Show a => Show (MyMaybe a) where
    show (Pure a)   = "Just " ++ show a
    show (Impure _) = "Nothing"

do a <- myJust 1
   b <- myJust 41
   return (a + b)
    
data Const e a = Const e

instance Functor (Const e) where
    fmap _ (Const e) = Const e
    
type MyEither e a = Term (Const e) a

myLeft :: e -> MyEither e a
myLeft = Impure . Const

myRight :: a -> MyEither e a
myRight = Pure

instance (Show e, Show a) => Show (MyEither e a) where
    show (Pure a)           = "Right " ++ show a
    show (Impure (Const e)) = "Left " ++ show e
    
do a <- myRight 4
   b <- if a > 5 then myRight 40 else myLeft 1337
   return (b + 2)
   
do a <- myRight 4
   b <- if a < 5 then myRight 40 else myLeft 1337
   return (b + 2)

42

Just 42

Left 1337

Right 42

In [72]:
data Incr t = Incr Int t

instance Functor Incr where
    fmap f (Incr n t) = Incr n (f t)

data Recall t = Recall (Int -> t)

instance Functor Recall where
    fmap f (Recall r) = Recall (f . r)

inject :: (g :<: f) => g (Term f a) -> Term f a
inject = Impure . inj

incr :: (Incr :<: f) => Int -> Term f ()
incr i = inject (Incr i (Pure ()))

recall :: (Recall :<: f) => Term f Int
recall = inject (Recall Pure)

tick :: Term (Recall :+: Incr) Int
tick = do y <- recall
          incr 1
          return y